In [ ]:
# This code was created with the help of AI
from faker import Faker
import numpy as np
import pandas as pd

fake = Faker()

In [ ]:
np.random.seed(42)
Faker.seed(42)

In [ ]:
n_records = 27549
average_courses_per_student = 3.1

In [ ]:
# Calculate the number of unique students based on the average number of courses per
# student
n_unique_students = int(n_records / average_courses_per_student)

# Generate unique student data
unique_students = pd.DataFrame(
    {
        "Student ID": [
            f"ID{''.join(np.random.choice(list('0123456789'), 9))}"
            for _ in range(n_unique_students)
        ],
        "Student Alternate ID": np.NaN,
        "Student Name": [fake.name() for _ in range(n_unique_students)],
        "Student E-mail": [fake.email() for _ in range(n_unique_students)],
        "Tags": [
            np.random.choice(["Active", "Pending", "Completed", "Enrolled", ""])
            for _ in range(n_unique_students)
        ],
        "Classification": np.random.choice(
            ["Freshman", "Sophomore", "Junior", "Senior", "Graduate"], n_unique_students
        ),
        "Major": np.random.choice(
            [
                "Computer Science",
                "Business Administration",
                "Psychology",
                "Education",
                "Engineering",
            ],
            n_unique_students,
        ),
        "Cumulative GPA": np.clip(np.random.normal(3, 0.5, n_unique_students), 0, 4),
    }
)

In [ ]:
# Simulate the number of courses per student using a distribution (you can adjust this
# distribution to better fit your data)
num_courses_per_student_distribution = {
    1: 0.2114755722921415,
    2: 0.21167376870478644,
    3: 0.2546823902487365,
    4: 0.1274402933306907,
    5: 0.09315231394311763,
    6: 0.05054008522445744,
    7: 0.020909721534040235,
    8: 0.008621543950054504,
    9: 0.005450401347735606,
    10: 0.005648597760380537,
    11: 0.0016846695074819145,
    12: 0.004657615697155881,
    13: 0.0027747497770290357,
    14: 0.0004954910316123278,
    15: 9.909820632246556e-05,
    16: 0.0004954910316123278,
    17: 0.00019819641264493112,
}
num_courses_per_student = np.random.choice(
    list(num_courses_per_student_distribution.keys()),
    n_unique_students,
    p=list(num_courses_per_student_distribution.values()),
)

In [ ]:
# Replicate each student entry based on the number of courses they're taking
replicated_students = unique_students.loc[
    unique_students.index.repeat(num_courses_per_student)
].reset_index(drop=True)

In [ ]:
# Generate Categories
def select_categories() -> str:
    categories = []

    def _include_category(chance: float = 0.9) -> bool:
        """Randomly decide to include a category or not"""
        return np.random.rand() < chance

    campus_options = ["On-line", "Main Campus", "Satellite Campus"]
    graduated_options = [  # Mutually exclusive with certain other categories
        "Graduated: Yes",
        "Graduated: No",
    ]
    level_options = [
        "Undergraduate",
        "Graduate",
    ]
    hold_types = ["Hold: Financial", "Hold: Academic", "Hold: Administrative"]
    comp_rate_options = ["Completion Rate: >= 66.67%", "Completion Rate: < 66.67%"]
    start_term_options = [
        f"Start Term: {season} {year}"
        for year in range(2000, 2024)
        for season in ["Fall", "Winter", "Spring", "Summer"]
    ]
    term_status_options = [  # Mutually exclusive with "Graduated: Yes"
        "Term Status: Registered",
        "Term Status: Not Registered",
    ]
    fafsa_options = [  # Mutually exclusive with "Graduated: Yes"
        "FAFSA: Yes",
        "FAFSA: No",
    ]

    # Build up categories one-by-one, randomly deciding which will be included.
    if _include_category():
        categories.append(
            f"Campus: {np.random.choice(campus_options, p=[0.5, 0.4, 0.1])}"
        )
    if _include_category():
        categories.append(np.random.choice(graduated_options, p=[0.1, 0.9]))
    if _include_category():
        categories.append(np.random.choice(level_options))
    if _include_category(chance=0.10):  # Only 10% chance of showing a Hold
        categories.append(np.random.choice(hold_types))
    if _include_category():
        categories.append(np.random.choice(comp_rate_options, p=[0.75, 0.25]))
    if _include_category():
        total_weight = sum(range(1, len(start_term_options) + 1))
        start_term_weights = [
            i / total_weight for i in range(1, len(start_term_options) + 1)
        ]
        categories.append(np.random.choice(start_term_options, p=start_term_weights))

    if "Graduated: Yes" not in categories:
        # We can only be here if you do NOT show "Graduated: Yes"
        if _include_category():
            categories.append(np.random.choice(term_status_options))
        if _include_category():
            categories.append(np.random.choice(fafsa_options))

    return ", ".join(categories)


replicated_students["Categories"] = [
    select_categories() for _ in range(len(replicated_students))
]

In [ ]:
# Course info
# Generate course-specific information for each enrollment
course_depts = ["MTH", "ENG", "HSC", "EDL"]
course_numbers = [
    f"{dept}-{str(i).zfill(3)}" for dept in course_depts for i in range(1, 101)
]  # Assuming 100 courses per department
course_titles = [
    fake.unique.sentence(nb_words=3) for _ in range(len(course_numbers))
]  # Unique course titles
replicated_students["Course Number"] = np.random.choice(
    course_numbers, len(replicated_students)
)
replicated_students["Course Title"] = [
    course_titles[course_numbers.index(cn)]
    for cn in replicated_students["Course Number"]
]
replicated_students["Section"] = [
    "".join(np.random.choice(list("0123456789"), 5))
    for _ in range(len(replicated_students))
]
replicated_students["Instructors"] = [
    (
        f"{fake.last_name()},"
        f" {fake.first_name()} (ID{''.join(np.random.choice(list('0123456789'), 9))})"
        f" <{fake.email()}>"
    )
    for _ in range(len(replicated_students))
]

In [ ]:
# Enrollment info
dropped = np.random.choice(["Yes", "No"], p=[0.24, 0.76], size=n_records)
dropped_dates = [
    fake.date_between(start_date="-1y", end_date="today") if drop == "Yes" else None
    for drop in dropped
]

In [ ]:
# Add other enrollment-specific information (e.g., grades, attendance) in a similar
# manner
replicated_students["Midterm Grade"] = np.random.choice(
    ["A", "B", "C", "D", "F"], len(replicated_students)
)
replicated_students["Final Grade"] = np.random.choice(
    ["A", "B", "C", "D", "F"], len(replicated_students)
)
replicated_students["Total Progress Reports"] = np.random.poisson(
    0.4, len(replicated_students)
)
replicated_students["Absences"] = np.random.poisson(0.5, len(replicated_students))

In [ ]:
# Schedule information
replicated_students["Start Date"] = [
    fake.date_between(start_date="-3m", end_date="+3m")
    for _ in range(len(replicated_students))
]
replicated_students["End Date"] = [
    sd + pd.Timedelta(weeks=16) for sd in replicated_students["Start Date"]
]
replicated_students["Start Time"] = [
    f"{hour}:00 AM CT"
    for hour in np.random.choice(range(8, 12), len(replicated_students))
]
replicated_students["End Time"] = [
    f"{hour + np.random.choice([1, 2, 3])}:00 PM CT"
    for hour in np.random.choice(range(1, 5), len(replicated_students))
]
replicated_students["Class Days"] = [
    ", ".join(
        np.random.choice(
            ["M", "T", "W", "R", "F", "Sa"], size=np.random.randint(1, 3), replace=False
        )
    )
    for _ in range(len(replicated_students))
]

In [ ]:
# Create DataFrame
# and ensure that the dataframe is not longer than `n_records`
df = replicated_students.iloc[:n_records]

In [ ]:
df